# **DSFM Exercise**: Sentiment Analysis - NLP, Text Embedding (SOLUTIONS)

Creator: [Data Science for Managers - EPFL Program](https://www.dsfm.ch)  
Source:  [https://github.com/dsfm-org/code-bank.git](https://github.com/dsfm-org/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

-------------

## Overview

Sentiment analysis is a challenging subject in machine learning. People express their emotions in a way that can be very ambiguous for both humans and computers. In this demo, we analyze sentiments of a set of IMDB movie reviews. The dataset consists of review texts as well as a binary sentiment label (1: positive, 0: negative). 

<img src="http://barnraisersllc.com/wp-content/uploads/2017/01/Sentiment-Analysis.jpg" width="500" height="500" align="center"/>

Image source: http://barnraisersllc.com/wp-content/uploads/2017/01/Sentiment-Analysis.jpg

Dataset source: *Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). "Learning Word Vectors for Sentiment Analysis." The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).*

-------------

## **Part 0**: Setup

In [ ]:
# Put all import statements at the top of your notebook

# Standard imports
import pandas as pd
import numpy  as np
from bs4 import BeautifulSoup 
import re

# Data science packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection         import KFold, GridSearchCV, cross_val_score
from sklearn.ensemble                import RandomForestClassifier
from sklearn.pipeline                import Pipeline
from sklearn.model_selection         import train_test_split

# Neural networks
from tensorflow.keras.models                 import Sequential, load_model
from tensorflow.keras.layers                 import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text     import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils.vis_utils import model_to_dot

# Text processing packages
import nltk
import nltk.data
from nltk.corpus import stopwords 
from nltk.stem   import SnowballStemmer
from gensim.models import doc2vec, word2vec

# Visualization packages
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython.display import SVG

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

%matplotlib inline


In [ ]:
# Set constants 

# Set a seed for replication
SEED = 10

# Set performance metric
SCORE = 'accuracy'

In [ ]:
# Nested cross validation helper function
def nested_cv(X, y, est_pipe, p_grid, p_score, n_splits_inner = 3, n_splits_outer = 3, n_cores = 1, seed = 0):

    # Cross-validation schema for inner and outer loops (stratified if it is a classification)
    inner_cv = KFold(n_splits = n_splits_inner, shuffle = True, random_state = seed)
    outer_cv = KFold(n_splits = n_splits_outer, shuffle = True, random_state = seed)
    
    # Grid search to tune hyper parameters
    est = GridSearchCV(estimator = est_pipe, param_grid = p_grid, cv = inner_cv, scoring = p_score, n_jobs = n_cores)

    # Nested CV with parameter optimization
    nested_scores = cross_val_score(estimator = est, X = X, y = y, cv = outer_cv, scoring = p_score, n_jobs = n_cores)
    
    print('Average score: %0.4f (+/- %0.4f)' % (nested_scores.mean(), nested_scores.std() * 1.96))
    
    return nested_scores.mean(), nested_scores.std() * 1.96

# Define a function to split a review into clean list of words
def review_to_wordlist(review):
    
    review_text = BeautifulSoup(review).get_text()
   
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    words = review_text.lower().split()
    
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(w) for w in words]
    
    return(words)

# Define a function to split a review into parsed sentences, where each sentence is a word list
def review_to_sentences(review, tokenizer):
    
    raw_sentences = tokenizer.tokenize(review.strip())  
    sentences = []
    for raw_sentence in raw_sentences:      
        if len(raw_sentence) > 0:           
            sentences.append( review_to_wordlist( raw_sentence ))
   
    return sentences

# Function to average all of the word vectors in a given paragraph
def makeFeatureVec(words, model, num_features):
    
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype='float32')
    nwords = 0.
     
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

# Given a set of reviews (each one a list of words), calculate 
# the average feature vector for each one and return a 2D numpy array
def getAvgFeatureVecs(reviews, model, num_features):
    
    # Initialize a counter
    counter = 0
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype='float32')
     
    # Loop through the reviews
    for review in reviews:
       
       # Print a status message every 1000th review
        if counter%1000 == 0:
            print ('Review %d of %d' % (counter, len(reviews)))
       
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       
        # Increment the counter
        counter = counter + 1
        
    return reviewFeatureVecs

## **Part 1**: EDA

In the first part, we load the `sentiment_data.tsv` dataset. The dataset consists of `id`, `sentiment`, and `review` columns.

Hint: This is a *tab*-separated file, but we can still use pandas' `read_csv` function. 


**Q 1**: Load the data. What shape does it have?

In [ ]:
# Load data
data = pd.read_csv('sentiment_data.tsv', header=0, delimiter='\t', quoting=3)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.columns

**Q 2**: Extract the observation at index 0. How does the review text look?

In [ ]:
# A sample observation
print ('id: \t\t', data['id'][0])
print ('sentiment: \t', data['sentiment'][0], '\n')
print (data['review'][0])

**Q 3**: Plot the distribution of the target and plot a word cloud.

In [ ]:
# Distribution of target
pd.DataFrame.hist(data,column='sentiment')

In [ ]:
# Visualize word distributions using the word cloud library

# Concatenate all rows of review column
text = data['review'].str.cat(sep=' ')

# Generate a word cloud image
wordcloud = WordCloud(width=1600, height=800).generate(text)

# Display the generated image
wordcloud = WordCloud(max_font_size=60).generate(text)
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

**Q 4**: Check for missing values and drop rows containing missing values (if any).

In [ ]:
# Check for missing values and drop corresponding rows, if any
data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
print(data.shape)

## **Part 2**: Text pre-processing

In the second part, we pre-process the raw review text in the following steps:

1. Remove HTML tags
2. Keep only alphabetical terms
3. Lowercase all words
4. Remove stop words
5. Stem words

IMPORTANT: Use the review at index 0 for each step to see how it changes. We will pre-process all reviews in the next step.

**Q 1**: Remove HTML tags with the `BeautifulSoup` package.

In [ ]:
# Remove HTML tags 
data_no_html = BeautifulSoup(data['review'][0])  

print('Original:\n{}\n'.format(data['review'][0]))
print('Without HTML tags:\n{}\n'.format(data_no_html.get_text()))

**Q 2**: Keep only alphabetical terms/words.

In [ ]:
# Keep only alphabetical terms
data_no_digits = re.sub('[^a-zA-Z]', ' ', data_no_html.get_text())  
print('No digits:\n{}'.format(data_no_digits))

**Q 3**: Lowercase all words and separate text by blank spaces.

In [ ]:
# Lower case and separate into tokens
lower_case = data_no_digits.lower()        
words = lower_case.split(' ')
print('Lower-cased and token-separated:\n{}'.format(words))

**Q 4**: Remove stop words.

In [ ]:
# Download the stop words dataset of NLTK library
nltk.download('stopwords')

In [ ]:
# Remove stop words
print('Stop words in NLTK:\n{}\n'.format(stopwords.words('english')))
words_no_stop = [w for w in words if not w in stopwords.words('english')]
print('Without stop words:\n{}'.format(words_no_stop))

**Q 5**: Stem all words.

In [ ]:
# Stem words

stemmer = SnowballStemmer('english')

print('grows  --> {}'.format(stemmer.stem('grows')))
print('leaves --> {}'.format(stemmer.stem('leaves')))
print('fairly --> {}\n'.format(stemmer.stem('fairly')))

words_stemmed = [stemmer.stem(w) for w in words_no_stop]
print('Stemmed:\n{}'.format(words_stemmed))

**Q 6**: Consolidate all steps into a single function called `review_to_words`.

In [ ]:
# Function to convert a raw review to a string of words: 
# the input is a single string (a raw movie review) and
# the output is a single string (a preprocessed movie review)

def review_to_words(raw_review):

    # Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    
    # Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    
    # In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
     
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    
    # Stem words
    stem_words = [stemmer.stem(w) for w in meaningful_words]
    
    # Join the words back into one string separated by space and return the result
    return( " ".join( stem_words )) 

In [ ]:
# An example of text preprocessing

clean_review = review_to_words(data['review'][0])

print('Original:\n{}\n'.format(data['review'][0]))
print('Cleaned:\n{}'.format(clean_review))

**Q 7**: Pre-process all reviews with the new function.

In [ ]:
# Clean and parse all movie reviews
num_reviews = data['review'].size
clean_data_reviews = []

for i in range(0, num_reviews):
    clean_data_reviews.append(review_to_words(data['review'][i]))
    
    # If the index is evenly divisible by 1000, print a message to show progress
    if (i+1) % 1000 == 0:
        print('Review {} of {}\t{}%'.format(i+1, num_reviews, round((i+1)/num_reviews*100, 0)))
    
assert len(clean_data_reviews) == len(data), 'Error: the number of cleaned reviews does not match.'

## **Part 3**: Feature Extraction using TFIDF

In this short part, we transform the cleaned text into TFIDF representations.

**Q 1**: Transform cleaned reviews to TFIDF using the 5000 most frequent words in the corpus. What shape will the new representation have?

In [ ]:
# Transforming pre-processed reviews to bag of words (BOW) feature representation
vectorizer = TfidfVectorizer(max_features = 5000)
features = vectorizer.fit_transform(clean_data_reviews)
features = features.toarray()
print (features.shape)

In [ ]:
# Get feature names (each term is a feature)
vocab = vectorizer.get_feature_names()
print (vocab[0])

**Q 2**: Print the TFIDF values of the first 10 words in the vocabulary.

In [ ]:
features.shape

In [ ]:
# Print the first 10 vocabulary words and their corresponding TFIDF values

dist = np.sum(features, axis=0)
i = 0
for tag, count in zip(vocab, dist):
    print(tag.ljust(15), count)
    i = i + 1
    if i > 10 : 
        break

In [ ]:
len(vocab)

In [ ]:
np.sum(features, axis=0).shape

In [ ]:
# See the values of feature 0
features[0]

## **Part 4**: Binary classification using Random Forest with TFIDF Features

In this part, we will run a binary classification using Random Forest and the TFIDF features to predict the binary sentiment label.

**Q 1**: Extract the target vector from the data.

In [ ]:
# Define the target

target = data['sentiment']

**Q 2**: Fit a `RandomForestClassifier` to our data and test different parameter values for `n_estimators` (e.g. from 10 to 50). 

What nested cross validated accuracy do you get? How long does it take to test the different parameter values?

Hint 1: Use the `%%time` magic command at the beginning of the cell for timing.

Hint 2: Use the `nested_cv` helper function defined at the very top of this notebook. 

In [ ]:
%%time

# Define pipeline
estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = SEED)

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

acc_rf, std_rf = nested_cv(X = features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = SCORE, n_cores = -1)

## **Part 5**: Word Vectors - Word2Vec

A popular vectorization method for words is a technique known as Word2Vec, which is implemented in the `gensim` library. 

In Word2Vec each word is assigned a low-dimensional vector which is learnt under the assumption that words that are close to each other in a document are semantically related. Word2Vec can be used as a base for vectorize documents in low dimensions. You can read more about it here: https://cs224d.stanford.edu/lectures/CS224d-Lecture2.pdf.

**Q 1**: Extract all sentences using the `punkt` tokenizer in the `nltk` package. How many sentences are in the corpus?

Hint: Use the `review_to_sentences` helper function defined at the very top of this notebook to convert reviews to sentences.

In [ ]:
# Download the punkt tokenizer for sentence splitting
nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
# Extract all sentences from all reviews
sentences = []

for review in data['review']:
    sentences += review_to_sentences(review, tokenizer)

In [ ]:
# Check number of sentences
print (len(sentences))

In [ ]:
# Check the first sentence
print (sentences[0])

**Q 2**: Train the word vectors using the `Word2Vec` function in the `gensim` package. Generate word vectors with 300 dimensions.

In [ ]:
%%time
# Train word vectors

# Set values for various parameters
num_features   = 300  # word vector dimensionality                      
min_word_count = 40   # minimum word count                        
num_workers    = 16   # number of threads to run in parallel
context        = 10   # context window size                                                                                    

# Initialize and train the model 
print ('Training model...')
w2v_model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)
print('Done !')

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient
w2v_model.init_sims(replace=True)
 
# Save the model for later use. you can load it later using Word2Vec.load()
# model_name = 'w2v_imdb'
# model.save(model_name)

**Q 3**: Investigate how words relate to each other using the `doesnt_match` and `most_similar` functions on the trained Word2Vec model. What's the most similar word to "man"?

In [ ]:
# Identify the word which is less similar to other words in a set
w2v_model.doesnt_match("man woman child kitchen".split())

In [ ]:
w2v_model.doesnt_match("france england germany berlin".split())

In [ ]:
# Identify the words which are similar to a focal word

w2v_model.most_similar("man")

In [ ]:
w2v_model.most_similar("queen")

In [ ]:
w2v_model.most_similar("bad")

In [ ]:
# Corresponding word vector of an example word

w2v_model['flower']

### Paragraph vectors - Average Word Vectors

Even though the word vectors show good semantical properties, using them to get the same sort of properties from sentences is not straight forward. The simplest solution is to average word vectors of a document to come up with the same dimensional paragraph vector.

Hint 1: Use the `review_to_wordlist` helper function defined at the very top of this notebook. 

Hint 2: Use the `getAvgFeatureVecs` helper function defined at the very top of this notebook to average word vectors.

In [ ]:
# Calculate average feature vectors for review data, using the functions we defined above.
clean_data_reviews = []
for review in data['review']:
    clean_data_reviews.append( review_to_wordlist( review ))

w2v_features = getAvgFeatureVecs(clean_data_reviews, w2v_model, num_features)

**Q 4**: Fit a `RandomForestClassifier` to our data and test different parameter values for `n_estimators` (e.g. from 10 to 50). 

What nested cross validated accuracy do you get? How long does it take to test the different parameter values?

In [ ]:
%%time

# Define pipeline
estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = SEED)

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

acc_rfW2V, std_rfW2V = nested_cv(X = w2v_features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = SCORE, n_cores = -1)

## **Part 6**: Document Vectors - Doc2Vec

Another solution to obtain review vectors is to obtain them directly. Paragraph Vectors are treating each document as a word itself and obtains their vectors directly. You can read more about it here: https://cs.stanford.edu/~quocle/paragraph_vector.pdf . Again, the `gensim` library implements this method in the `doc2vec` class.

The first two cells below get rid of some "housekeeping" to prepare the data. Make sure the variable names match with your variable names.

In [ ]:
# Doc2Vec needs each review to be tagged with some sort of ids
# Here we tag each review with the 'id' field

tagged_clean_data_reviews = []
for uid, review in zip(data['id'], clean_data_reviews):
    tagged_clean_data_reviews.append(doc2vec.TaggedDocument(words=review, tags=['%s' % uid[1:-1]]))

In [ ]:
# An example of tagging

tagged_clean_data_reviews[0]

**Q 1**: Train the document vectors using the `Doc2Vec` function in the `gensim` package. Generate document vectors with 300 dimensions.

In [ ]:
%%time
# Train paragraph vectors

# Set values for various parameters
num_features   = 300  # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers    = 4    # Number of threads to run in parallel
context        = 10   # Context window size                                                                                    

# Initialize and train the model (this will take some time)
print('Training model...')
d2v_model = doc2vec.Doc2Vec(tagged_clean_data_reviews, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)
print('Done !')

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
d2v_model.init_sims(replace=True)
 
# Save the model for later use. You can load it later using Doc2Vec.load().
# d2v_model_name = 'w2v_imdb'
# d2v_model.save(d2v_model_name)

**Q 2**: Investigate how documents relate to each other using the `docvecs.similarity` function on the trained Doc2Vec model. How similar are documents d1 = '7759_3' and d2 = '5814_8'?

In [ ]:
# Corresponding document vector of a review given its id
d2v_model.docvecs['5814_8']

In [ ]:
# Create a set of D2V features of all reviews to be used in down-stream predictions 
d2v_features = d2v_model.docvecs
d2v_features[0]

In [ ]:
# Report the similarity of two reviews (based on cosine similarity) 
d2v_model.docvecs.similarity(d1='7759_3', d2='5814_8')

**Q 3**: Fit a `RandomForestClassifier` to our data and test different parameter values for `n_estimators` (e.g. from 10 to 50). Hint: use the `linspace` function in the `numpy` package to generate evenly spaced numbers in a given interval.

What nested cross validated accuracy do you get? How long does it take to test the different parameter values?

In [ ]:
%%time

# Define pipeline
estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = SEED)

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

acc_rfD2V, std_rfD2V = nested_cv(X = d2v_features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = SCORE, n_cores = -1)

## **SUMMARY OF ACCURACY SCORES**

In [ ]:
width       = 50
models      = ['Baseline', 'Random Forest', 'Random Forest + word2vec', 'Random Forest + doc2vec']
result_acc  = [0.5, acc_rf, acc_rfW2V, acc_rfD2V]
result_std  = [0,   std_rf, std_rfW2V, std_rfD2V]

print('', '=' * width, '\n', 'Summary of Accuracy Scores'.center(width), '\n', '=' * width)  
for i in range(len(models)):
    print(models[i].center(width-18), '{0:.4f}'.format(result_acc[i]), '+/-{0:.4f}'.format(result_std[i]))